Please make sure all the libraries used in the project are installed in your local machine if you are using Jupyter notebook

If not, please remove the # in the next line and run it

In [ ]:
# pip install numpy pandas sklearn matplotlib seaborn statsmodels

In [ ]:
# Import the libraries for data analysis, visualisation and Machine Learning Models
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from scipy import stats

In [ ]:
# Set the display or output in 5 decimal places
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
# Read the CSV file and store it into the varieble - father_son_df
father_son_df = pd.read_csv("Hong Kong Father-Son Height Dataset.csv")

In [ ]:
# Read the first 6 rows data
print(father_son_df.head(6))

The dataset conatins father's and son's height in float with will be verified later

In [ ]:
# Get the summary description of the father_son_df
print(father_son_df.info())

In [ ]:
# Preliminary check if all the data is float, no NaN and 0 indeed
# NaN Check
print('NA in the dataset:', father_son_df.isnull().any().any())
# Data = 0 check
print('0 in the dataset:', father_son_df.eq(0).any().any())
# Data Type Check
print('All data is float:', father_son_df.applymap(lambda x: isinstance(x, float), na_action='ignore').any().all())

All the data are in float and there is no missing data or height with 0 value. Data cleaning is not necessary here.

In [ ]:
# Statistics summaries of the data
print(father_son_df.describe())

In [ ]:
# Correlation of the two data set
print(father_son_df.corr())
print('\n',father_son_df.cov())
# Calutate the slope in linear regression by using closed form solution
print('\nBeta in the linear regression :', '%.5f' % (father_son_df.cov()['Father'].iloc[1]/father_son_df.cov()['Father'].iloc[0]) )

The correlation matrix indicates that the Father's and Son's height are positive correlation with correlation = 0.50076

If I use all the data for performaing simple linear regression, the beta will be 0.51381.

I am interested in how the son's height is affected by father's height.

Father's height will be considered as independent variable whereas Son's height will be considered as dependent variable

In [ ]:
# Simple Linear Regression in Scikit-Learn
li_reg = LinearRegression()
li_reg.fit(father_son_df['Father'].values.reshape(-1,1), father_son_df['Son'].values.reshape(-1,1))
print("Linear Regression Coefficient:", '%.5f' % li_reg.coef_[0][0])
print("Linear Regression Intercept:", '%.5f' % li_reg.intercept_[0])

By performing simple linear regression, the slope and interception is as of above

It suggests the relationship between father's and son's height can be represented as 

$$Son's Height = 87.14969 +0.51381*Father's Height$$

In [ ]:
# Visualise the data in scatter plot and the distribution
# Father are consider as input valiable while Son are consider as output varible
sns.set(style="ticks", context='notebook')
jointplot = sns.jointplot(x = "Father", y = "Son", data = father_son_df, kind = "reg", height = 10)
jointplot.fig.suptitle("Father's and Son's Height Scatterplot and Distribution", size = 20)
jointplot.set_axis_labels( "Father's height" , "Son's height")
jointplot.fig.tight_layout()
jointplot.fig.savefig("Father's and Son's Height Scatterplot and Distribution.png")

In [ ]:
# QQ Plot for checking the normal distribution assupmtion
sm.qqplot(father_son_df['Father'], line = 's', alpha = 0.5)
plt.title("Father's height Q-Q plot",  fontsize = 20)
plt.savefig("Father's height Q-Q plot.png")
plt.show()
sm.qqplot(father_son_df['Son'], line = 's', alpha = 0.5)
plt.title("Son's height Q-Q plot",  fontsize = 20)
plt.savefig("Son's height Q-Q plot.png")
plt.show()

Q-Q plots and the joint plot shows that Father's and Son's height may follow normal distribution and the homoscedasticity may hold with the fitted line

Now, I am interested in testing the model. If i use all the data for model fitting, there is no out sample for me to evaluation its performance. As a result, I split the data into training and testing set

Moreover, RANdom SAmple Consensus (RANSAC) Algorithm will be used for model comparison.

In [ ]:
# Reshape to 2D array from 1D array
x_value_raw, y_value_raw = father_son_df['Father'].values.reshape(-1,1), father_son_df['Son'].values.reshape(-1,1) 

In [ ]:
# Split testing, 20%, and training set, 80%,
x_train, x_test, y_train, y_test = train_test_split(x_value_raw,  y_value_raw, test_size = 0.2 , random_state = 0)

Model1 represents Simple Linear Regression

Model2 represents RANSAC

In [ ]:
# Linear Regression in Scikit-Learn
model1 = LinearRegression()
# Model fitting
model1.fit(x_train, y_train)

In [ ]:
# Robust Regression: RANdom SAmple Consensus (RANSAC) Algorithm, default parameters unchanged, setting random_state = 0 such that it generate same reset for each run given the dataset unchange
model2 = RANSACRegressor(random_state = 0)
# model fitting
model2.fit(x_train, y_train)

In [ ]:
# Seperated the outliers from the training data
inlier = model2.inlier_mask_
outlier = np.logical_not(inlier)

In [ ]:
# predict the dependent variable with given independent variable
model1_train_pred = model1.predict(x_train)
model2_train_pred = model2.predict(x_train)

In [ ]:
# Visualise the fitted models with training data
plt.figure(figsize=(10,10))
plt.scatter(x_train[inlier], y_train[inlier], c='blue', alpha=0.5, marker='o', label='Inliers')
plt.scatter(x_train[outlier], y_train[outlier], c='brown', alpha=0.5, marker='o', label='Outliers')
plt.plot(x_train, model2_train_pred, color='blue')
plt.plot(x_train, model1_train_pred, color='brown')
plt.title("Father's and Son's Height Scatterplot & Regession Lines",  fontsize = 20)
plt.xlabel("Father's height")
plt.ylabel("Son's height")
plt.legend(loc='upper left', labels=["Inliner", "Outliner", 'RANSAC', 'Linear Regression'])
plt.savefig("Father's and Son's Height Scatterplot & Regession Lines.png")
plt.show()

In [ ]:
# Pritting the coefficient in both models with training data
print("Linear Regression Coefficient: ", '%.5f' % model1.coef_[0][0])
print("Linear Regression Intercept: ", '%.5f' % model1.intercept_[0])
print("RANSAC Coefficient: " '%.5f' % model2.estimator_.coef_[0][0])
print("RANSAC Intercept: " '%.5f' % model2.estimator_.intercept_[0])

Two fitted line are cloesd to each other and the "outliners" considered by RANSAC may contain actual useful information.

Simple Linear Regression suggests the model $$Son's Height = 87.99503+0.50951*Father's Height$$

RANSAC suggests the model $$Son's Height = 86.92336+0.51331*Father's Height$$

## Performance Analysis

In [ ]:
# Performance Analyisis by using RMSE
print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_train, model1_train_pred)))
print("RANSAC RMSE:", np.sqrt(mean_squared_error(y_train, model2_train_pred)))

In [ ]:
# k-fold cross-validation RMSE score, k = 10
model1_score = cross_val_score(model1, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
model1_rmse_score = np.sqrt(-model1_score)
print("Linear Regression RMSE Mean:", model1_rmse_score.mean(), ", RMSE Std:", model1_rmse_score.std())

model2_score = cross_val_score(model2, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
model2_rmse_score = np.sqrt(-model2_score)
print("RANSAC RMSE Mean:", model2_rmse_score.mean(), ", RMSE Std:", model2_rmse_score.std())


In [ ]:
# Visualise the residal for linear model with training data
plt.figure(figsize=(12,8))
plt.scatter(x_train, model1_train_pred - y_train, c='blue', marker='o', label='Training data', alpha = 0.5)
plt.title("Linear Regression Residal Analysis",  fontsize = 20)
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin= min(x_value_raw) , xmax=max(x_value_raw), color='k')
plt.savefig("Linear Regression Residal Analysis Train.png")
plt.show()

In [ ]:
# Visualise the residal for RANSAC with training data
plt.figure(figsize=(12,8))
plt.scatter(x_train, model2_train_pred - y_train, c='blue', marker='o', label='Training data', alpha = 0.5)
plt.title("RANSAC Residal Analysis",  fontsize = 20)
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin= min(x_value_raw) , xmax=max(x_value_raw), color='k')
plt.savefig("RANSAC Residal Analysis Train.png")
plt.show()

RMSE suggests that the Simple Linear Regression outperform RANSAC in training data. The residal distribution is roughly the same and scatter around the fitted line. Homoscedasticity may not violate.

Now, fitting the test data into the training models

In [ ]:
# Evaluate the test set with fitted models
model1_test_pred = model1.predict(x_test)
model2_test_pred = model2.predict(x_test)

In [ ]:
# Visualise the fitted modes with test data
plt.figure(figsize=(10,10))
plt.scatter(x_test, y_test, c = 'grey', alpha=0.5, marker='o', label='Testing Set')
plt.plot(x_train, model2_train_pred, color='blue')
plt.plot(x_train, model1_train_pred, color='brown')
plt.title("Father's and Son's Height Scatterplot & Regession Lines",  fontsize = 20)
plt.xlabel("Father's height")
plt.ylabel("Son's height")
plt.legend(loc='upper left', labels=['Testing Set', 'RANSAC', 'Linear Regression'])
plt.savefig("Father's and Son's Height Scatterplot & Regession Lines Test.png")
plt.show()

In [ ]:
print("Linear Regression RMSE in Test set:", np.sqrt(mean_squared_error(y_test, model1_test_pred)))
print("RANSAC RMSE in Test set:",np.sqrt(mean_squared_error(y_test, model2_test_pred)))

In [ ]:
# Visualise the residal with test data set in linear regression model
plt.figure(figsize=(12,8))
plt.scatter(x_test, model1_test_pred - y_test, c='red', marker='*', label='Test data', alpha = 0.5)
plt.title("Linear Regression Residal Analysis",  fontsize = 20)
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin= min(x_value_raw) , xmax=max(x_value_raw), color='k')
plt.savefig("Linear Regression Residal Analysis Test.png")
plt.show()

In [ ]:
# Visualise the residal with test data in RANSAC
plt.figure(figsize=(12,8))
plt.scatter(x_test, model2_test_pred - y_test, c='red', marker='*', label='Test data', alpha = 0.5)
plt.title("RANSAC Residal Analysis",  fontsize = 20)
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin= min(x_value_raw) , xmax=max(x_value_raw), color='k')
plt.savefig("RANSAC Residal Analysis Test.png")
plt.show()

In [ ]:
#computer the 95% confidence interval for the generalization error 
ci = 0.95
se = (model1_test_pred - y_test) ** 2
print("95% confidence interval for linear regression generalization error:", np.sqrt(stats.t.interval(ci, len(se) - 1, loc=se.mean(), scale=stats.sem(se))) )

In [ ]:
#computer the 95% confidence interval for the generalization error 
se2 = (model2_test_pred - y_test) ** 2
print("95% confidence interval for RANSAC generalization error:", np.sqrt(stats.t.interval(ci, len(se2) - 1, loc=se2.mean(), scale=stats.sem(se2))) )

RMSE and 95% confidence interval may suggested RANSAC performs better than simple linear regression in test set.